In [7]:
from IPython.display import display, HTML
from io import StringIO
import requests
import json
import pandas as pd
%matplotlib inline

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [1]:
# The code was removed by DSX for sharing.

[Row(=1, CASE_STATUS='CERTIFIED-WITHDRAWN', EMPLOYER_NAME='UNIVERSITY OF MICHIGAN', SOC_NAME='BIOCHEMISTS AND BIOPHYSICISTS', JOB_TITLE='POSTDOCTORAL RESEARCH FELLOW', FULL_TIME_POSITION='N', PREVAILING_WAGE='36067', YEAR='2016', WORKSITE='ANN ARBOR, MICHIGAN', lon='-83.7430378', lat='42.2808256'),
 Row(=2, CASE_STATUS='CERTIFIED-WITHDRAWN', EMPLOYER_NAME='GOODMAN NETWORKS, INC.', SOC_NAME='CHIEF EXECUTIVES', JOB_TITLE='CHIEF OPERATING OFFICER', FULL_TIME_POSITION='Y', PREVAILING_WAGE='242674', YEAR='2016', WORKSITE='PLANO, TEXAS', lon='-96.6988856', lat='33.0198431'),
 Row(=3, CASE_STATUS='CERTIFIED-WITHDRAWN', EMPLOYER_NAME='PORTS AMERICA GROUP, INC.', SOC_NAME='CHIEF EXECUTIVES', JOB_TITLE='CHIEF PROCESS OFFICER', FULL_TIME_POSITION='Y', PREVAILING_WAGE='193066', YEAR='2016', WORKSITE='JERSEY CITY, NEW JERSEY', lon='-74.0776417', lat='40.7281575'),
 Row(=4, CASE_STATUS='CERTIFIED-WITHDRAWN', EMPLOYER_NAME='GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY OF TOMKINS PLC', SOC_NAME='CHIEF

In [19]:
df_data_3.head(10)

[Row(=1, CASE_STATUS='CERTIFIED-WITHDRAWN', EMPLOYER_NAME='UNIVERSITY OF MICHIGAN', SOC_NAME='BIOCHEMISTS AND BIOPHYSICISTS', JOB_TITLE='POSTDOCTORAL RESEARCH FELLOW', FULL_TIME_POSITION='N', PREVAILING_WAGE='36067', YEAR='2016', WORKSITE='ANN ARBOR, MICHIGAN', lon='-83.7430378', lat='42.2808256'),
 Row(=2, CASE_STATUS='CERTIFIED-WITHDRAWN', EMPLOYER_NAME='GOODMAN NETWORKS, INC.', SOC_NAME='CHIEF EXECUTIVES', JOB_TITLE='CHIEF OPERATING OFFICER', FULL_TIME_POSITION='Y', PREVAILING_WAGE='242674', YEAR='2016', WORKSITE='PLANO, TEXAS', lon='-96.6988856', lat='33.0198431'),
 Row(=3, CASE_STATUS='CERTIFIED-WITHDRAWN', EMPLOYER_NAME='PORTS AMERICA GROUP, INC.', SOC_NAME='CHIEF EXECUTIVES', JOB_TITLE='CHIEF PROCESS OFFICER', FULL_TIME_POSITION='Y', PREVAILING_WAGE='193066', YEAR='2016', WORKSITE='JERSEY CITY, NEW JERSEY', lon='-74.0776417', lat='40.7281575'),
 Row(=4, CASE_STATUS='CERTIFIED-WITHDRAWN', EMPLOYER_NAME='GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY OF TOMKINS PLC', SOC_NAME='CHIEF

In [20]:
(df_data_3[['CASE_STATUS', 'SOC_NAME']]).show()
#use this to make a chart with two indexes

+-------------------+--------------------+
|        CASE_STATUS|            SOC_NAME|
+-------------------+--------------------+
|CERTIFIED-WITHDRAWN|BIOCHEMISTS AND B...|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|          WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|          WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|CERTIFIED-WITHDRAWN|  FINANCIAL MANAGERS|
|CERTIFIED-WITHDRAWN|    CHIEF EXECUTIVES|
|          CERTIFIED|    CHIEF EXECUTIVES|
|          CERTIFIED|    CHIEF EXECUTIVES|
+----------

In [24]:
df_data_3.select('SOC_NAME').count()

3002458

In [5]:
df_data_2['SOC_NAME'].value_counts().count()

2132

In [6]:
testdata=df_data_2.copy()

In [7]:
testdata['SOC_NAME']=df_data_2['SOC_NAME'].str.lower()
testdata['EMPLOYER_NAME']=df_data_2['EMPLOYER_NAME'].str.lower()
testdata['JOB_TITLE']=df_data_2['JOB_TITLE'].str.lower()

Here, we transform some strings to lowercase. This is because there are duplicate entries in the dataset which in both upper and lower.
This increases redundancy

In [8]:
testdata['SOC_NAME'].value_counts()

computer systems analysts                                       506523
software developers, applications                               414716
computer programmers                                            398546
computer occupations, all other                                 177870
software developers, systems software                            83856
management analysts                                              66873
financial analysts                                               53540
accountants and auditors                                         52822
mechanical engineers                                             44236
network and computer systems administrators                      40087
database administrators                                          38982
market research analysts and marketing specialists               37737
electronics engineers, except computer                           36574
operations research analysts                                     34260
electr

There is still alot of redundancy we can exploit. We can generalize these strings remove specalized strings into more general form.
For example 'software engineer, senior' needs to be reduced to 'software engineer'.
This also applies to the other columns with string attributes.

In [9]:
testdata['SOC_NAME'].value_counts().count()

1585

In [10]:
df = pd.DataFrame({'SOC_NAME': testdata['SOC_NAME'].value_counts().index, 'Count':testdata['SOC_NAME'].value_counts().values})
#df['Counts'] = df.groupby(['SOC_NAME'])['Count'].transform('count') #I don't remember what I was trying to do here.
df = df.set_index(['SOC_NAME'])
df

,Count
SOC_NAME,
computer systems analysts,506523
"software developers, applications",414716
computer programmers,398546
"computer occupations, all other",177870
"software developers, systems software",83856
management analysts,66873
financial analysts,53540
accountants and auditors,52822
mechanical engineers,44236


In [11]:
df['New_Name'] = ''

In [ ]:
df

,Count,New_Name
SOC_NAME,,
computer systems analysts,506523,
"software developers, applications",414716,
computer programmers,398546,
"computer occupations, all other",177870,
"software developers, systems software",83856,
management analysts,66873,
financial analysts,53540,
accountants and auditors,52822,
mechanical engineers,44236,


In [ ]:
%timeit
for index, row in testdata.iterrows():
    if(str(row['SOC_NAME']).endswith('*')):
        row['SOC_NAME']=str(row['SOC_NAME'])[:-1]
    if not (str(row['SOC_NAME']).endswith('s')):
        row['SOC_NAME']=str(row['SOC_NAME'])+'s'
    #if row=='software developers, appllications': #in progress
        #print(row)

In [ ]:
(testdata.loc[(testdata['SOC_NAME']=='software developers, appllications')]) #an example of a query

This person messed up the SOC_NAME